    This code load the annotations, the output direction for the saved images and the name of the test set from which the images are visualized
    # TUE Project 2023 - Visualize Inference Results

    This Jupyter Notebook is used to visualize the inference results of a Detectron2 model on a test set of images. The notebook assumes that the model has already been trained and the inference results have been saved to a JSON file.

    The notebook contains the following cells:

    - **Cell 0:** This markdown cell that describes the purpose and contents of the notebook.
    - **Cell 1:** Imports the necessary libraries and loads the annotations and test set information.
    - **Cell 2:** Visualizes random images from the test set with ground truth annotations.
    - **Cell 3:** Visualizes the predictions and saves the images to the output directory.
    - **Cell 4:** Creates dictionaries for visualizing false positives, false negatives, and class false positives.
    - **Cell 5:** Computes the IoU between each ground truth box and predicted boxes and creates dictionaries for true positives, false positives, false negatives, and class false positives.
    - **Cell 6:** Saves the visualizations of false positives, false negatives, and class false positives to the output directory.

    To use this notebook, make sure that the necessary libraries are installed and that the paths to the annotations, test set, and output directory are correct. Then, run each cell in order to visualize the inference results and create the necessary dictionaries for further analysis.
    

In [ ]:
# Imports
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
import random
import os
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

from detectron2.data.datasets import register_coco_instances
import json
import os

annotation_path = "/mnt/d/TIL_Melanoma_train_database/cell_segmentation/open_source_dataset_tiles_final/til_melanoma.json" # path to annotation file from 01_coco_creator_5fold
with open(annotation_path, "r") as f:
    data = json.load(f)
    images = data['images']

# Directory where the original annotation resides
annotation_dir = os.path.dirname(annotation_path)

# Predefined folder to save the visualized images
output_dir = '/mnt/d/TIL_Melanoma_train_database/cell_segmentation/detectron2_inference'

# Load the predictions
predictions_file = '/home/mschuive/detectron2/output/inference/coco_instances_results.json'


import matplotlib.pyplot as plt
test_set_name = 'temp_val_annotations_stratified_fold_4'
test_set_name_json = test_set_name + '.json'
if test_set_name in DatasetCatalog.list():
    DatasetCatalog.remove(test_set_name)
register_coco_instances(str(test_set_name), {}, os.path.join(annotation_dir, (test_set_name_json)), "/mnt/d/TIL_Melanoma_train_database/cell_segmentation/tiles_1024_TIL_dataset_paper/redone/")


# Get dataset dictionaries
dataset_dicts = DatasetCatalog.get(test_set_name)



    Visualize random images from test set with ground truth, images are not saved

In [ ]:
# Get dataset dictionaries
dataset_dicts = DatasetCatalog.get(test_set_name)

# Visualize some samples
for d in random.sample(dataset_dicts, 5):  # Adjust the number to display as needed
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get(test_set_name), scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    # plt.figure(figsize=(10,10))
    # plt.imshow(vis.get_image()[:, :, ::-1])
    # plt.title(d["file_name"])
    # plt.axis("off")
    # plt.show()


    Visualize predictions and save the images 
    NB update the predictions file localization 

In [ ]:
import json
import torch

with open(predictions_file, 'r') as f:
    predictions = json.load(f)

from detectron2.structures import BoxMode, Instances
import numpy as np

def visualize_predictions_on_image(image, predictions_for_image):
    """
    Visualizes predictions on the image.
    
    Args:
    - image (np.ndarray): Image on which to visualize the predictions.
    - predictions_for_image (list[dict]): List of predictions for the image.
    
    Returns:
    - visualized_image (np.ndarray): Image with visualized predictions.
    """
    # Convert the predictions in Detectron2's Instances format
    instances = Instances(image_size=image.shape[:2])
    bbox_list = [pred["bbox"] for pred in predictions_for_image]
    bbox_list_xyxy_abs = []
    for bbox in bbox_list:
        bbox = BoxMode.convert(bbox, from_mode=BoxMode.XYWH_ABS, to_mode=BoxMode.XYXY_ABS)
        bbox_list_xyxy_abs.append(bbox)
    instances.pred_boxes = bbox_list_xyxy_abs
    cat_list_new = []
    cat_list = [pred["category_id"] for pred in predictions_for_image]
    for cat in cat_list:
        if cat == 1:
            cat = 0
        elif cat == 2:
            cat = 1
        else:
            cat = 2
        cat_list_new.append(cat)
    cat_tensor = torch.tensor(cat_list_new)
    instances.pred_classes = cat_tensor
    instances.scores = [pred["score"] for pred in predictions_for_image]
    instances.pred_masks_rle = [pred["segmentation"]["counts"]for pred in predictions_for_image]
    
    visualizer = Visualizer(image[:, :, ::-1], metadata=MetadataCatalog.get(test_set_name).set(thing_classes=['immune_cell', 'tumor', 'other']), scale=1.2)
    vis = visualizer.draw_instance_predictions(predictions=instances)
    return vis.get_image()[:, :, ::-1]

# Get unique image_ids from the predictions
image_ids = list(set([pred["image_id"] for pred in predictions]))

# For each unique image_id, visualize the predictions
for image_id in image_ids:
    # Get the image
    image_dict = next(item for item in dataset_dicts if item["image_id"] == image_id)
    if image_dict is None:
        print(f"No item found for image_id {image_id}")
        continue  # Skip this iteration and continue with the next image_id
    image = cv2.imread(image_dict["file_name"])
    
    # Extract predictions for this image_id
    predictions_for_image = [pred for pred in predictions if pred["image_id"] == image_id]
    
    # Visualize the predictions on the image
    visualized_image = visualize_predictions_on_image(image, predictions_for_image)
    
    # Save the image with highlighted false negatives to the output directory
    output_path = os.path.join(output_dir, f"predictions{image_id}.png")
    cv2.imwrite(output_path, visualized_image)
    
    plt.figure(figsize=(10,10))
    plt.imshow(visualized_image[:, :, ::-1])
    plt.title(f"Predictions for Image {image_id}")
    plt.axis("off")
    plt.show()




    Code which creates dictionary's for visualizing false positive, false negative and class false positive

In [ ]:
from detectron2.structures import pairwise_iou
from detectron2.structures import Boxes, BoxMode
import json
import torch


with open(predictions_file, 'r') as f:
    predictions = json.load(f)

# Get unique image_ids from the predictions
image_ids = list(set([pred["image_id"] for pred in predictions]))


class_id_to_label = {
    0: ["immune_cell", (0, 0, 255)], # blue for immune cell 
    1: ["tumor", (255,0 ,0)] , # red for tumor
    2: ["other", (0, 255, 0)], # green for other
    'none' : ["none", (0, 0, 0)]
}

class_false_negative = []
# IoU threshold to determine if a prediction matches a ground truth
upper_threshold = 0.4  # Example value for upper threshold
lower_threshold = 0.0  # Example value for lower threshold

true_positive = []
false_positive = []
false_negative = []
class_false_positive = []

# For each unique image_id, visualize the predictions
for image_id in image_ids:

    # Get the image
    gt_image_dict = next(item for item in dataset_dicts if item["image_id"] == image_id)
    image = cv2.imread(gt_image_dict["file_name"])

    # Extract predictions for this image_id
    predictions_for_image = [pred for pred in predictions if pred["image_id"] == image_id]
    for entry in predictions_for_image:
        # Define a mapping dictionary
        category_id_mapping = {1: 0, 2: 1, 3: 2}
        entry['category_id'] = category_id_mapping[entry['category_id']]
        bbox_xyxy = BoxMode.convert(entry['bbox'], from_mode=BoxMode.XYWH_ABS, to_mode=BoxMode.XYXY_ABS)
        entry['bbox_abs'] = bbox_xyxy
        # Extract the bbox_abs values and add their sum to total_sum

  

    # Extract ground truth bounding boxes and classes for this image_id
    ground_truth_for_image = gt_image_dict["annotations"]
    for entry in ground_truth_for_image:
        entry['category_id'] = entry['category_id']
        bbox_xyxy = BoxMode.convert(entry['bbox'], from_mode=BoxMode.XYWH_ABS, to_mode=BoxMode.XYXY_ABS)
        entry['bbox_abs'] = bbox_xyxy

    gt_boxes = [ann["bbox_abs"] for ann in gt_image_dict["annotations"]]    

    gt_classes = [ann["category_id"] for ann in gt_image_dict["annotations"]]

    pred_boxes = [pred["bbox_abs"] for pred in predictions_for_image]

    pred_classes = [pred["category_id"] for pred in predictions_for_image]
    pred_score = [pred["score"] for pred in predictions_for_image]

    # Compute IoU between each ground truth box and predicted boxes
    # Convert lists to tensors
    tensor_gt_boxes = torch.tensor(gt_boxes)
    tensor_pred_boxes = torch.tensor(pred_boxes)

    # Create Boxes objects from tensors
    boxes_gt = Boxes(tensor_gt_boxes)
    boxes_pred = Boxes(tensor_pred_boxes)

    iou_matrix = pairwise_iou(boxes_gt, boxes_pred)

    above_threshold_indices = torch.nonzero(iou_matrix > upper_threshold, as_tuple=True)
    below_threshold_indices = torch.nonzero((iou_matrix < upper_threshold) & (iou_matrix > lower_threshold), as_tuple=True)
    above_0_indices = torch.nonzero(iou_matrix > 0, as_tuple=True)

    boxesgt_above_threshold = boxes_gt.tensor[above_threshold_indices[0]]
    boxespred_above_threshold = boxes_pred.tensor[above_threshold_indices[1]]
    
    boxesgt_below_threshold = boxes_gt.tensor[below_threshold_indices[0]]
    boxespred_below_threshold = boxes_pred.tensor[below_threshold_indices[1]]

    # Assuming gt_classes and pred_classes are dictionaries with box indices as keys
    # Subselect classes for boxes above the threshold
    gt_classes_above_threshold = {i: gt_classes[i] for i in above_threshold_indices[0]}
    pred_classes_above_threshold = {i: pred_classes[i] for i in above_threshold_indices[1]}


    # Subselect classes for boxes below the threshold
    gt_classes_below_threshold = {i: gt_classes[i] for i in below_threshold_indices[0]}
    pred_classes_below_threshold = {i: pred_classes[i] for i in below_threshold_indices[1]}

    gt_classes_above_threshold_list = list(gt_classes_above_threshold.values())
    pred_classes_above_threshold_list = list(pred_classes_above_threshold.values())

    gt_classes_below_threshold_list = list(gt_classes_below_threshold.values())
    pred_classes_below_threshold_list = list(pred_classes_below_threshold.values())

    all_overlap_indices = torch.nonzero(iou_matrix > 0, as_tuple=True)

    # Find the indices of false positive boxes by removing the indices of true positives
    false_positive_indices = set(all_overlap_indices[1].tolist()) - set(above_threshold_indices[1].tolist())
    false_positive_indices = list(false_positive_indices)

    boxespred_false_positive = boxes_pred.tensor[false_positive_indices]
    pred_classes_false_positive = {i: pred_classes[i] for i in false_positive_indices}
    pred_classes_false_positive_list = list(pred_classes_false_positive.values())




    annotations_tp = []
    annotations_class_fp = []
    annotations_fp = []

    true_positive.append(
        {'image_id': image_id,
        'filename': gt_image_dict["file_name"],
        'annotations': annotations_tp,
        'type': 'True Positive'
        }
    )

    class_false_positive.append(
        {'image_id': image_id,
        'filename': gt_image_dict["file_name"],
        'annotations': annotations_class_fp,
        'type': 'Class False Positive'
        }
    )

    for i in range(len(gt_classes_above_threshold_list)):
        if gt_classes_above_threshold_list[i] == pred_classes_above_threshold_list[i]:
            annotations_tp.append(
                {
                    # 'image_id': image_id,
                    # 'filename': gt_image_dict["file_name"],
                    'gt_class': gt_classes_above_threshold_list[i],
                    'pred_class': pred_classes_above_threshold_list[i],
                    'gt_box': boxesgt_above_threshold[i].tolist(),
                    'pred_box': boxespred_above_threshold[i].tolist(),
                    'pred_score': pred_score[i]
                }
            )
           
            
        else:
            annotations_class_fp.append(
                {
                    # 'image_id': image_id,
                    # 'filename': gt_image_dict["file_name"],
                    'gt_class': gt_classes_above_threshold_list[i],
                    'pred_class': pred_classes_above_threshold_list[i],
                    'gt_box': boxesgt_above_threshold[i].tolist(),
                    'pred_box': boxespred_above_threshold[i].tolist(),
                    'pred_score': pred_score[i]
                    
                }
            )   
        
    # Update the false positive annotations
    false_positive.append(
        {
            'image_id': image_id,
            'filename': gt_image_dict["file_name"],
            'annotations': annotations_fp,
            'type': 'False Positive'
        }   
    )

    for i in range(len(boxespred_false_positive)):
        annotations_fp.append(
            {
                'pred_class': pred_classes_false_positive_list[i],
                'pred_box': boxespred_false_positive[i].tolist(),
                'pred_score': pred_score[i],
                'gt_class': 'none',
            }
        )
 

    Code for visualizing true positive, false positive and false negative

In [ ]:
import cv2
import matplotlib.pyplot as plt

def draw_predictions(class_id_to_label, output_dir, dictionary):
  """Highlights predictions on the image.

  Args:
    class_id_to_label: A dictionary mapping class IDs to labels and colors.
    output_dir: The directory where the output images should be saved.
    mismatch_info: A dictionary containing information about the false negative,
      including the predicted bounding box, predicted class ID, ground truth class ID,
      and image filename.
  """


  for im in dictionary:
    # Load the image if not already loaded
    image = cv2.imread(im['filename'])

    for ann in im['annotations']:
      pred_box = ann['pred_box']
      pred_class_id = ann['pred_class']
      gt_class_id = ann['gt_class']
      pred_score = ann['pred_score']
      
      # get label text and color
      pred_label_text = class_id_to_label[pred_class_id][0]
      gt_label_text = class_id_to_label[gt_class_id][0]
      label_color = class_id_to_label[gt_class_id][1]  # Use ground truth class color

      # Construct label text
      label = f"FN: GT-{gt_label_text}, PD-{pred_label_text} {pred_score:.2f}"

      pred_box = [int(coord) for coord in pred_box]


      # Draw rectangle and put text on the image
      cv2.rectangle(image, (pred_box[0], pred_box[1]), (pred_box[2], pred_box[3]), label_color, 2)
      cv2.putText(image, label, (int(pred_box[0]), int(pred_box[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_color, 2)


     # Display the image with highlighted false negatives in Jupyter Notebook
    plt.figure(figsize=(10,10))
    plt.imshow(image[:, :, ::-1])
    plt.title(f"{im['type']}, {im['image_id']}")
    plt.axis("off")
    plt.show()
    cv2.imwrite(os.path.join(output_dir, f"{im['type']}_{im['image_id']}.png"), image)

draw_predictions(class_id_to_label, output_dir, class_false_positive)
draw_predictions(class_id_to_label, output_dir, true_positive)
draw_predictions(class_id_to_label, output_dir, false_positive)